In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100%}</style>"))

In [1]:
import re
import shutil
import calendar
import psycopg2
import calendar
import datetime
import pandas as pd
import win32com.client as MyWinCOM
from dateutil.relativedelta import relativedelta

In [2]:
dealerReinsurer = {'Ellison':'Triple C - RC', \
                   'Abernethy Chrysler Jeep Dodge': 'Abernethy Chrysler Jeep Dodge - WHS', \
                   'Approved Auto of America': 'Approved Auto of America - WH', \
                   'Aston Martin Summit': 'Aston Martin Summit',\
                   'Attica':'Attica', \
                   'BNM':'Wahoo - RC', \
                   'Bedford': 'Bedford', \
                   'Bell Mitsubishi':'Bell Mitsubishi - WH', \
                   'Pellegrino Auto Sales, LLC':'Pellegrino Auto Sales - WH',\
                   'Five Star Motors':'PPT – WH', \
                   'Roselle Motors': 'Roselle Motors - WH', \
                   '1800 Warranty': '1800W - WH', \
                   'ServiceContract': 'SC.Com', \
                   'AMG Auto|Parkwood Auto Mall': 'AMG/Parkwood Group - WH', \
                   'Cars & Credit Masters': 'Cars & Credit Masters - WH', \
                   'Kiplin': 'Cars & Credit Masters - WH', \
                   'Eagle Motors': 'PPT - WH', \
                   'Heards Enterprise Inc.': 'PPT - WH', \
                   'National Advance Auto': 'PPT - WH', \
                   'Risner Auto Group': 'PPT - WH', \
                   'Tri City Motors': 'DOWC Provider Services',
                   'Wiltshire Auto Sales, LLC': '.', \
                   'Crown': 'Crown Auto Sales and Finance-WHS', \
                   'Garner Autos LLC':'PPT - WH', \
                   'Heards Enterprise Inc.|Eagle Motors|National Advance Auto|Risner Auto Group|Iron City Imports, LLC|Wiltshire Auto Sales, LLC|Auto Arena|Tri City Motors':'.', \
                   'Jersey Car Direct, LLC':'Jersey Car Direct - WH',\
                   'Flawless Auto Mall':'Flawless Auto Mall - WH', \
                   'M and I Motors Inc': 'M and I - WH', \
                   'On The Road Again': 'PPT', \
                   'Iron City Imports, LLC':'PPT - WH', \
                   'Flemington Ford Lincoln': 'Flemington - WHS|All WHS Cession', \
                   'Flemington Chevrolet, Buick, GMC, Cadillac':'Flemington - WHS|All WHS Cession',\
                   'Flemington Dodge  Chyrsler  Jeep':'Flemington - WHS|All WHS Cession',\
                   'Flemington Infiniti':'Flemington - WHS|All WHS Cession',\
                   'Flemington Subaru':'Flemington - WHS|All WHS Cession',\
                   'Flemington VW Porsche Audi':'Flemington - WHS|All WHS Cession',\
                   'Flemington BMW':'Flemington - WHS|All WHS Cession', \
                   'Clinton Honda':'Flemington - WHS|All WHS Cession', \
                   'Land Rover Princeton':'Flemington - WHS|All WHS Cession', \
                   '.': 'PPT - WH', \
                   'The Luxury Autohaus': 'TL Assurance Reinsurance', \
                   'Park Place of NJ': 'Park Place of NJ', \
                   'Peak Auto Sales':'Peak Auto Sales - WH', \
                   'Miloschs Palace': 'Miloschs Palace', \
                   'Illinois Valley': 'Illinois Valley', \
                   '^VIP Honda': 'VIP Honda', \
                   '^Bridgewater Acura': 'Bridgewater Acura', \
                   'M and I|Crown|Birmingham Auto|Garner|P&M Auto|Lanier Automotive|Mardi Gras Motors|Oldham Motor|Peak Auto Sales|Perfect Ride|Wright Automotive': '- WH|-WH',\
                   '^Century Auto Group Inc':'.*',\
                   "Dan O''Brien|Infiniti of Hanover": '.', \
                   'Parkwood Auto': '- WH|', \
                   'Forthright Auto': 'Forthright Auto - WH', \
                   'Hiley':'Optimus Group - WH', \
                   'Jersey City Kia|Kia Of Englewood':'Jersey City Kia', \
                   'Mighty Motors':'Mighty Motors',\
                   'Auto Holding':'Drexel - RC', \
                   'U.S. Auto':'U.S. Auto WH', \
                   'El Paso Honda|Poe Toyota': '.', \
                   }

BHPHList = ['.', 'Heards Enterprise Inc.|Eagle Motors|National Advance Auto|Risner Auto Group|Iron City Imports, LLC|Wiltshire Auto Sales, LLC|Auto Arena|Tri City Motors', \
            'Ellison', 'BNM', 'Cars & Credit Masters', 'Eagle Motors', 'Heards Enterprise Inc.', \
            'National Advance Auto', 'Risner Auto Group', 'Tri City Motors', 'Wiltshire Auto Sales, LLC']

In [3]:
dealerProduct = {'PPT':'1st Nations Auto Sales & ServiceTECH_DO_NC|1st Nations Auto Sales & ServiceKEY_GPS|1st Nations Auto Sales & ServiceLTW_GPS|1st Nations Auto Sales & ServiceTECH_GPS|1st Nations Auto Sales & ServiceVSC_GPS|216 Auto SalesTECH_AO_NC|Autoxperts, IncExtdcare_GPS|Birmingham Auto AuctionTECH_AO_NC|Birmingham Auto AuctionVSC_AO_NC|Columbus Auto SourceLTW_SPS|Columbus Auto SourceVSC_SPS|Diamond Auto Group LLCLTW_SPS|Diamond Auto Group LLCVSC_SPS|Eagle MotorsTECH_SPS|Eagle MotorsVSC_SPS|Exclusive Auto Sales LLCLTW_SPS|Five Star MotorsDCA_DO|G&B Auto SalesLTW_GPS|G&B Auto SalesTECH_GPS|G&B Auto SalesVSC_GPS|Garner Autos LLCKEY_AO_NC|Garner Autos LLCLTW_IADA|Garner Autos LLCTECH_AO_NC|Garner Autos LLCVSC_AO_NC|Garner Autos LLCVSC_GPS|Heards Enterprise Inc.LTW_IADA|Heards Enterprise Inc.TECH_AO_NC|Heards Enterprise Inc.VSC_AO_NC|Heards Enterprise Inc.TECH_FCIC|Heards Enterprise Inc.TECH_GPS|Heards Enterprise Inc.VSC_FCIC|Iron City Imports, LLCLTW_GPS|Iron City Imports, LLCVSC_GPS|K and K AutomotiveLTW_IADA|Killough Auto Sales LLCextdcare|Killough Auto Sales LLCTECH_AO_NC|Killough Auto Sales LLCVSC_AO_NC|L&L AutosLTW_IADA|L&L AutosTECH_FCIC|LakeSideImports.net Inc.LTW_IADA|LakeSideImports.net Inc.TECH_AO_NC|LakeSideImports.net Inc.VSC_AO_NC|Lanier Automotive LLCBNDL_FCIC|Lanier Automotive LLCLTW_IADA|Lanier Automotive LLCTECH_FCIC|Lanier Automotive LLCVSC_FCIC|Lanier Automotive LLCVSC_GPS|Luxury Imports Auto SalesDCA_DO|Mardi Gras Motors LLCLTW_SPS|Mardi Gras Motors LLCTECH_SPS|Mardi Gras Motors LLCVSC_SPS|Marketplace Mazdaextdcare|Marketplace MazdaVSC_FCIC|McAdenville MotorsLTW_GPS|McAdenville MotorsTECH_GPS|McAdenville MotorsVSC_GPS|MS Coast Autobrokers GPT LLC-GulfportLTW_SPS|National Advance Auto SalesLTW_IADA|National Advance Auto SalesTECH_AO_NC|Nice Clean CarzLTW_IADA|Nice Clean CarzVSC_FCIC|Oldham Motor Co of Wake ForestLTW_GPS|Oldham Motor Co of Wake ForestTECH_GPS|Oldham Motor Co of Wake ForestVSC_GPS|Oldham Motor CompanyLTW_GPS|Oldham Motor CompanyTECH_GPS|Oldham Motor CompanyVSC_GPS|On The Road Again Inc.LTW_IADA|On The Road Again Inc.TECH_FCIC|On The Road Again Inc.VSC_FCIC|One Stop Auto SalesLTW_IADA|One Stop Auto SalesVSC_FCIC|Peak Auto SalesLTW_IADA|Peak Auto SalesLTW_IADA|Perfect RideTECH_DO_NC|Perfect RideLTW_GPS|Perfect RideTECH_GPS|Perfect RideVSC_GPS|Physicians Auto GroupLTW_DO_NC2|Platinum Used CarsLTW_DO_NC2|Platinum Used CarsplusCAREppm|PLM MotorsportsBNDL_FCIC|PLM MotorsportsLTW_IADA|PLM MotorsportsplusCAREppm|PLM MotorsportsCW_FCIC|PLM MotorsportspcPPM_AO|PLM MotorsportsTECH_FCIC|PLM MotorsportsVSC_FCIC|Risner Auto GroupLTW_SPS|Risner Auto GroupTECH_SPS|Risner Auto GroupVSC_SPS|Roberts Auto Sales - HartselleLTW_IADA|Summit MotorcarsLTW_SPS|Summit MotorcarsTECH_SPS|Summit MotorcarsVSC_SPS|Thornton Motors LLCTECH_FCIC|Thornton Motors LLCVSC_FCIC|Top Cars Inc.TECH_AO_NC|Top Cars Inc.VSC_AO_NC|Tri City MotorsTECH_AO_NC|Tri City MotorsVSC_AO_NC|Trophy FordLTW_AO_NC|Trophy FordpcPPM_AO|Trophy FordTCW_AO|Trophy FordTECH_AO_NC|Trophy FordTECH_FCIC|Trophy FordVSC_AO_NC|Trophy FordVSC_FCIC|Watkins Auto SalesLTW_IADA|Watkins Auto SalesVSC_AO_NC|West 40 Buy-Pay HereLTW_DO_NC2|West 40 Buy-Pay HereVSC_SPS|Wiltshire Auto Sales, LLCTECH_AO_NC|Wiltshire Auto Sales, LLCTECH_FCIC|Wiltshire Auto Sales, LLCVSC_AO_NC|Wright Automotive LLCKEY_GPS|Wright Automotive LLCLTW_GPS|Wright Automotive LLCTECH_GPS|Wright Automotive LLCVSC_GPS|Xtreme Motors Plus, Inc.VSCM_SPS|Xtreme Motors Plus, Inc.LTW_SPS|Xtreme Motors Plus, Inc.VSC_SPS|Columbus Powersports LLCextdcare|Columbus Powersports LLCExtdcare_SPS|Columbus Powersports LLCVSC_SPS', \
                 'ServiceContract':'ServiceContract.com RatingVSCSUB_GPSA|ServiceContract.com Region 2VSCSUB_GPSA|ServiceContract.com RatingVSCSUB_DPSA|ServiceContract.com Region 2SC_VSC_GPS|ServiceContract.com RatingSC_VSC_DPS|ServiceContract.com Region 1VSCSUB_SPSA|ServiceContract.com Region 4VSCSUB_APSA', \
                 'Flemington':'Clinton HondaDOBUNDLE|Clinton HondaPDR D|Clinton HondaPDR D|Clinton HondaplusCAREppm|Clinton HondardsWRAP|Clinton HondardsWRAP|Clinton HondascPPM|Clinton HondascPPM|Clinton HondaWDS D|Clinton HondaWDS D|Clinton HondaDOWC_GAP|Clinton HondaKey D|Clinton HondaKey D|Clinton HondaEWT D|Clinton HondaEWT D|Ditschman/Flemington Ford LincolnDOBUNDLE|Ditschman/Flemington Ford LincolnDOBUNDLE|Ditschman/Flemington Ford LincolnEWT D|Ditschman/Flemington Ford LincolnPDR D|Ditschman/Flemington Ford LincolnPDR D|Ditschman/Flemington Ford LincolnplusCAREppm|Ditschman/Flemington Ford LincolnplusCAREppm|Ditschman/Flemington Ford LincolnrdsWRAP|Ditschman/Flemington Ford LincolnscPPM|Ditschman/Flemington Ford LincolnscPPM|Ditschman/Flemington Ford LincolnWDS D|Ditschman/Flemington Ford LincolnWDS D|Ditschman/Flemington Ford LincolnDOWC_GAP|Ditschman/Flemington Ford LincolnKey D|Ditschman/Flemington Ford LincolnKey D|Ditschman/Flemington Ford LincolnEWT D|Ditschman/Flemington Ford LincolnEWT D|Flemington BMWDOBUNDLE|Flemington BMWDOBUNDLE|Flemington BMWDOWC_GAP|Flemington BMWKey D|Flemington BMWKey D|Flemington BMWEWT D|Flemington BMWEWT D|Flemington BMWEWT D|Flemington BMWEWT D|Flemington BMWPDR D|Flemington BMWPDR D|Flemington BMWrdsWRAP|Flemington BMWrdsWRAP|Flemington BMWWDS D|Flemington BMWWDS D|Flemington Chevrolet, Buick, GMC, CadillacDOBUNDLE|Flemington Chevrolet, Buick, GMC, CadillacDOBUNDLE|Flemington Chevrolet, Buick, GMC, CadillacEWT D|Flemington Chevrolet, Buick, GMC, CadillacPDR D|Flemington Chevrolet, Buick, GMC, CadillacPDR D|Flemington Chevrolet, Buick, GMC, CadillacplusCAREppm|Flemington Chevrolet, Buick, GMC, CadillacrdsWRAP|Flemington Chevrolet, Buick, GMC, CadillacrdsWRAP|Flemington Chevrolet, Buick, GMC, CadillacscPPM|Flemington Chevrolet, Buick, GMC, CadillacWDS D|Flemington Chevrolet, Buick, GMC, CadillacWDS D|Flemington Chevrolet, Buick, GMC, CadillacDOWC_GAP|Flemington Chevrolet, Buick, GMC, CadillacKey D|Flemington Chevrolet, Buick, GMC, CadillacKey D|Flemington Chevrolet, Buick, GMC, CadillacEWT D|Flemington Chevrolet, Buick, GMC, CadillacEWT D|Flemington Dodge  Chyrsler  JeepDOBUNDLE|Flemington Dodge  Chyrsler  JeepDOBUNDLE|Flemington Dodge  Chyrsler  JeepEWT D|Flemington Dodge  Chyrsler  JeepPDR D|Flemington Dodge  Chyrsler  JeepPDR D|Flemington Dodge  Chyrsler  JeepplusCAREppm|Flemington Dodge  Chyrsler  JeeprdsWRAP|Flemington Dodge  Chyrsler  JeeprdsWRAP|Flemington Dodge  Chyrsler  JeepWDS D|Flemington Dodge  Chyrsler  JeepWDS D|Flemington Dodge  Chyrsler  JeepDOWC_GAP|Flemington Dodge  Chyrsler  JeepKey D|Flemington Dodge  Chyrsler  JeepKey D|Flemington Dodge  Chyrsler  JeepEWT D|Flemington Dodge  Chyrsler  JeepEWT D|Flemington InfinitiDOBUNDLE|Flemington InfinitiDOBUNDLE|Flemington InfinitiEWT D|Flemington InfinitiPDR D|Flemington InfinitiPDR D|Flemington InfinitiplusCAREppm|Flemington InfinitirdsWRAP|Flemington InfinitiWDS D|Flemington InfinitiWDS D|Flemington InfinitiDOWC_GAP|Flemington InfinitiKey D|Flemington InfinitiKey D|Flemington InfinitiEWT D|Flemington InfinitiEWT D|Flemington SubaruDOBUNDLE|Flemington SubaruDOBUNDLE|Flemington SubaruEWT D|Flemington SubaruPDR D|Flemington SubaruPDR D|Flemington SubaruplusCAREppm|Flemington SubarurdsWRAP|Flemington SubaruWDS D|Flemington SubaruWDS D|Flemington SubaruDOWC_GAP|Flemington SubaruKey D|Flemington SubaruKey D|Flemington SubaruEWT D|Flemington SubaruEWT D|Flemington VW Porsche AudiDOBUNDLE|Flemington VW Porsche AudiDOBUNDLE|Flemington VW Porsche AudiPDR D|Flemington VW Porsche AudiPDR D|Flemington VW Porsche AudirdsWRAP|Flemington VW Porsche AudiRHT_DO_FCNC|Flemington VW Porsche AudiWDS D|Flemington VW Porsche AudiWDS D|Flemington VW Porsche AudiDOWC_GAP|Flemington VW Porsche AudiKey D|Flemington VW Porsche AudiKey D|Flemington VW Porsche AudiEWT D|Land Rover PrincetonDOBUNDLE|Land Rover PrincetonPDR D|Land Rover PrincetonPDR D|Land Rover PrincetonplusCAREppm|Land Rover PrincetonrdsWRAP|Land Rover PrincetonWDS D|Land Rover PrincetonWDS D|Land Rover PrincetonDOWC_GAP|Land Rover PrincetonKey D|Land Rover PrincetonKey D|Land Rover PrincetonEWT D|Land Rover PrincetonEWT D', \
                 'Jack Daniels': 'Jack Daniels Audi USR360S_GPS|Jack Daniels Audi USRCW_GPS|Jack Daniels Audi USRGAP_GPS|Jack Daniels Audi USRKEY_GPS|Jack Daniels Audi USRLC_GPS|Jack Daniels Audi USRPDR_GPS|Jack Daniels Audi USRRDS_GPS|Jack Daniels Audi USRTW_GPS|Jack Daniels Audi USRVSC_GPS|Jack Daniels Audi of Paramus360S_GPS|Jack Daniels Audi of ParamusCW_GPS|Jack Daniels Audi of ParamusGAP_GPS|Jack Daniels Audi of ParamusKEY_GPS|Jack Daniels Audi of ParamusLC_GPS|Jack Daniels Audi of ParamusPDR_GPS|Jack Daniels Audi of ParamusTW_GPS|Jack Daniels Audi of ParamusVSC_GPS|Jack Daniels KiaCW_GPS|Jack Daniels KiaGAP_GPS|Jack Daniels KiaKEY_GPS|Jack Daniels KiaLC_GPS|Jack Daniels KiaEWT_GPS|Jack Daniels KiaPDR_GPS|Jack Daniels KiaplusCAREppm|Jack Daniels KiaTW_GPS|Jack Daniels KiaVSC_GPS|Jack Daniels Porsche360S_GPS|Jack Daniels PorscheCW_GPS|Jack Daniels PorscheKEY_GPS|Jack Daniels PorscheEWT_GPS|Jack Daniels PorschePDR_GPS|Jack Daniels PorscheRDS_GPS|Jack Daniels PorscheTW_GPS|Jack Daniels PorscheVSC_GPS|Jack Daniels Volkswagen360S_GPS|Jack Daniels VolkswagenCW_GPS|Jack Daniels VolkswagenGAP_GPS|Jack Daniels VolkswagenKEY_GPS|Jack Daniels VolkswagenLC_GPS|Jack Daniels VolkswagenPDR_GPS|Jack Daniels VolkswagenTW_GPS|Jack Daniels VolkswagenVSC_GPS', \
                 'BNM':'BNM AutosLTW_APS|BNM AutosLTW_IADA|BNM AutosLTW_IADA|BNM AutosTECH_AO_NC|BNM AutosTECH_AO_NC|BNM AutosTECH_APS|BNM AutosTECH_GPS|BNM AutosVSC_AO_NC|BNM AutosVSC_AO_NC|BNM AutosVSC_APS', \
                 'M and I': 'M and I Motors IncBNDL_GPS|M and I Motors IncCW_GPS|M and I Motors IncExtdcare_GPS|M and I Motors IncKEY_GPS|M and I Motors IncLTW_GPS|M and I Motors IncPCMP_GPS|M and I Motors IncTECH_GPS|M and I Motors IncVSC_GPS', \
                 'Cars & Credit Masters': 'Cars & Credit Masters- HumbleVSC_GPS|Cars & Credit Masters- HoustonVSC_GPS|Cars & Credit Masters - Hou SW FWYVSC_GPS|Cars & Credit Masters- Forest Lane 4520VSC_GPS|Cars & Credit Masters- JoshuaVSC_GPS|Cars & Credit Masters- Garland Ave BVSC_GPS|Cars & Credit Masters- Forest Lane 4045VSC_GPS', \
                 'World Class': 'World Class Motors LLC.BNDL_DO_FCNC|World Class Motors LLC.CW_DO_FCNC|World Class Motors LLC.EXTNDCARE_DO|World Class Motors LLC.KEY_DO_FCNC|World Class Motors LLC.LTW_DO_NC|World Class Motors LLC.LTW_IADA|World Class Motors LLC.PDR_DO_FCNC|World Class Motors LLC.plusCAREppm|World Class Motors LLC.RHT_DO_FCNC|World Class Motors LLC.TECH_DO_NC|World Class Motors LLC.VSC_DOWC|World Class Motors LLC.WDS_DO_FCNC', \
                 'Car Shoppe':'The Car ShoppeLTW_DO_NC|The Car ShoppeLTW_DO_NC|The Car ShoppeLTW_DO_NC2|The Car ShoppeplusCAREppm|The Car ShoppeTECH_DO_NC|The Car ShoppeVSC_DOWC', \
                 'Starfleet':'Starfleet CarsTECH_SPS|Starfleet CarsVSC_SPS',\
                 'Santa Cruz Motors':'Santa Cruz MotorsLTW_DO_NC2', \
#                  'Columbus Powersports LLC': 'Columbus Powersports LLCextdcare|Columbus Powersports LLCExtdcare_SPS|Columbus Powersports LLCVSC_SPS', \
                 'Peak Auto':'Peak Auto SalesTECH_DO_NC|Peak Auto SalesVSC_DOWC|Peak Auto SalesCW_DO_FCNC', \
                 'Forthright':'Forthright AutoVSC_GPS|Forthright AutoLTW_DO_NC|Forthright AutoLTW_DO_NC2', \
                 'Autoxperts':'Autoxperts, IncExtdcare_GPS', \
                 'The Luxury Autohaus':'The Luxury AutohausBNDL_FCIC|The Luxury AutohausLTW|The Luxury AutohausplusCAREppm|The Luxury AutohausTECH_FCIC|The Luxury AutohaustechVSC_DO|The Luxury AutohausVSC_DOWC|The Luxury AutohausVSC_FCIC|The Luxury AutohausVSC2'
                }

BHPHList = ['.', 'Heards Enterprise Inc.|Eagle Motors|National Advance Auto|Risner Auto Group|Iron City Imports, LLC|Wiltshire Auto Sales, LLC|Auto Arena|Tri City Motors', \
            'Ellison', 'BNM', 'Cars & Credit Masters', 'Eagle Motors', 'Heards Enterprise Inc.', \
            'National Advance Auto', 'Risner Auto Group', 'Tri City Motors', 'Wiltshire Auto Sales, LLC']

In [3]:
dealerList = ['Parkwood Auto']
    
#     ['1st Nations Auto Sales & Service','216 Auto Sales',


#     'Bell Mitsubishi','Pellegrino Auto Sales, LLC','Five Star Motors','Roselle Motors','1800 Warranty', 'ServiceContract','AMG Auto']
#     'Jack Daniels', 'Park Place of NJ', 'The Luxury Autohaus']
#     'M and I|Crown|Birmingham Auto|Garner|P&M Auto|Lanier Automotive|Mardi Gras Motors|Oldham Motor|Peak Auto Sales|Perfect Ride|Wright Automotive']
#     'Heards Enterprise Inc.|Eagle Motors|National Advance Auto|Risner Auto Group|Iron City Imports, LLC|Wiltshire Auto Sales, LLC|Auto Arena|Tri City Motors']
#     'Flemington Ford Lincoln','Flemington Chevrolet, Buick, GMC, Cadillac','Flemington Dodge  Chyrsler  Jeep', 'Flemington Infiniti','Flemington Subaru','Flemington VW Porsche Audi', \
#               'Flemington BMW','Clinton Honda', 'Land Rover Princeton']
# dealerList = [sub.replace('PPT', 'Heards Enterprise Inc.|Eagle Motors|National Advance Auto|Risner Auto Group|Iron City Imports, LLC|Wiltshire Auto Sales, LLC|Auto Arena|Tri City Motors') for sub in dealerList]
year = '2020'
month = '9'

### Pull data from pgSQL and Create spreadsheets

#### Create the report based on dealer and product code

In [5]:
conn = psycopg2.connect(dbname='warehouse', user='chris', password='dowc_123', host='3.221.58.110', port='5432')
print('PgSQL Connected')

for dealer in dealerList:
    # SQL Query
    cancel_sql = """SELECT DISTINCT
                        "Dealer", "CashTransactionId", "SentDate"::text, "ReceivedDate"::text, "PostedDate"::TEXT,
                        "CurrentContractStatus", "DisbursementType", "ContractNumber", "Product", -1*sum("Amount"), "EffectiveDate"::TEXT
                    FROM
                        chris."tbl_WHCancel" whc
                    WHERE
                        concat("Dealer", "ProductCode") ~* '{}'
                        AND "PostedDate" IS NOT NULL
                        AND CASE
                                WHEN "CancelMethod" = 'FULL REFUND' THEN "DisbursementType" ~* 'reserve'
                                ELSE "DisbursementType" = 'Reserve'
                            END
                    GROUP BY
                        1,2,3,4,5,6,7,8,9,11
                    ORDER BY
                        5,1,8;""".format(dealerProduct[dealer])

    contract_sql = """SELECT
                            vr."Dealer", vr."CashTransactionId", vr."EffectiveDate"::text, vr."SentDate"::text, vr."PostedDate"::TEXT,
                            vr."TransactionTypeName", vr."DisbursementType", vr."ContractNumber", vr."Product", SUM(vr."Amount") AS "Reserve", vr."ProductCode", vr."ProductType"
                        FROM
                            chris."tbl_Reserve" vr
                        WHERE
                            concat("Dealer", "ProductCode")  ~* '{}'
                            AND vr."TransactionTypeName" !~* 'Cancel|Suspend|Denied|Pending Reinstatement'
                            AND vr."PostedDate" IS NOT NULL
                            AND vr."ReceivedDate" IS NOT NULL
                            AND vr."DisbursementType" ~* 'reserve'
                        GROUP BY
                            1,2,3,4,5,6,7,8,9,11,12
                        ORDER BY
                            5,1,8;""".format(dealerProduct[dealer])

    claim_sql = """SELECT DISTINCT
                        vc."Dealer", vc."CashTransactionId", vc."SentDate"::text, vc."ReceivedDate"::text, vc."PostedDate"::text, vc."TC", vc."ClaimNumber",
                        vc."Status", vc."SubStatus", vc."Contract", vc."Product", vc."Amount", vc."ProductCode"
                    FROM
                        chris."tbl_Claim" vc
                    WHERE
                        concat("Dealer", "ProductCode")  ~* '{}'
                        AND vc."SubStatus" ~* 'FULLY PAID'
                        AND vc."PostedDate" IS NOT NULL
                    ORDER BY
                        5,1,10;""".format(dealerProduct[dealer])

     #  Read data into Dataframe
    cancel = pd.read_sql_query(cancel_sql, conn)
    contract = pd.read_sql_query(contract_sql, conn)
    claim = pd.read_sql_query(claim_sql, conn)
    
    # Select the date to be used for the postedDate
    dt = datetime.date(int(year), int(month), 1)
    dt = dt + relativedelta(months=1) - relativedelta(days=1)
    maxDate = str(dt)
    
    # Get the aggregate earning data   
    earning_sql = """SELECT
                        CASE
                            WHEN p."PostedDate" < '2020-1-1' THEN date_part('year', p."PostedDate")::TEXT
                            ELSE CONCAT(date_part('year', p."PostedDate") , ' ', date_part('month', p."PostedDate"))
                        END AS "PostedDate", sum(er."EarnedAmount"::NUMERIC) AS "Amount"
                    FROM
                        ssasearnedreserve er
                    JOIN ssascontractdimension con
                        ON con."ContractId" = er."fxContractId"
                    JOIN ssasdealerdimension deal
                        ON deal."DealerId" = con."fxSoldByDealerId"
                    LEFT JOIN ssaspostinghistorydimension p
                        ON p."PostingHistoryId" = er."fxPostingHistoryId"
                    WHERE
                        concat(deal."Name", con."ProductCode") ~* '{}'
                        AND p."PostedDate" <= '{}'
                    GROUP BY
                        1
                    ORDER BY
                        1""".format(dealerProduct[dealer], maxDate)
    
    earning = pd.read_sql_query(earning_sql, conn)
    
    # Get the detailed earning data
    earningDetails_sql = """SELECT
                            deal."Name", con."ProductCode", sum(er."EarnedAmount"::NUMERIC) AS "Amount"
                        FROM
                            ssasearnedreserve er
                        JOIN ssascontractdimension con
                            ON con."ContractId" = er."fxContractId"
                        JOIN ssasdealerdimension deal
                            ON deal."DealerId" = con."fxSoldByDealerId"
                        LEFT JOIN ssaspostinghistorydimension p
                            ON p."PostingHistoryId" = er."fxPostingHistoryId"
                        WHERE
                            concat(deal."Name", con."ProductCode") ~* '{}'
                            AND p."PostedDate" <= '{}'
                        GROUP BY
                            1,2
                        HAVING
                            sum(er."EarnedAmount"::NUMERIC) <> 0
                        ORDER BY
                            1""".format(dealerProduct[dealer], maxDate)
    earningDetails = pd.read_sql_query(earningDetails_sql, conn)
    
    dealerEarning = earningDetails[['Name', 'Amount']].groupby('Name', as_index=False, sort=True).sum()
    productEarning = earningDetails[['ProductCode', 'Amount']].groupby('ProductCode', as_index=False).sum()
    
    # Used to count the ITD number of the product
    countContract = contract[['ProductType', 'ContractNumber']][~contract['ContractNumber'].isin(cancel.ContractNumber)].groupby('ProductType', as_index=False).count()
    product = countContract.rename(columns = {'ContractNumber': 'Count'})
    product.sort_values(by = ['Count'], ascending=False, inplace=True)

    # Create the dealer name used in the report
    if dealer == 'Heards Enterprise Inc.|Eagle Motors|National Advance Auto|Risner Auto Group|Iron City Imports, LLC|Wiltshire Auto Sales, LLC|Auto Arena|Tri City Motors' \
        or dealer == 'M and I|Crown|Birmingham Auto|Garner|P&M Auto|Lanier Automotive|Mardi Gras Motors|Oldham Motor|Peak Auto Sales|Perfect Ride|Wright Automotive':
        dealerName = 'PPT'
    elif dealer == "Dan O''Brien|Infiniti of Hanover":
        dealerName = 'Dan Obrien'
    else:
        dealerName = dealer
        
    # Create new file for the dealer's data
    new_file = 'C:/Users/Huijie Qu/Documents/Python Scripts/' + year + '.' + month + ' ' + dealerName + '.xlsx'
    bhphPath = 'C:/Users/Huijie Qu/Documents/Python Scripts/BHPH/2020.8 BHPH Reserve.xlsm'
    
    if dealer in BHPHList:
        shutil.copy2("C:/Users/Huijie Qu/Documents/Python Scripts/PPT BHPH.xlsx", new_file)
    else:
        shutil.copy2("C:/Users/Huijie Qu/Documents/Python Scripts/PPT.xlsx", new_file)

    NameOfDealer_sql = """SELECT "Name", "Number" FROM ssasdealerdimension s WHERE "Name" ~* '{}'""".format(dealerName)
    NameOfDealer_query = pd.read_sql_query(NameOfDealer_sql, conn)
    
    if NameOfDealer_query.empty:
        NameOfDealer = dealerName
        NumberOfDealer = ''
    else:
        NameOfDealer = NameOfDealer_query.Name[0]
        NumberOfDealer = NameOfDealer_query.Number[0]

    xl = MyWinCOM.Dispatch('Excel.Application')
    wb = xl.Workbooks.Open(new_file)
    ws = wb.Worksheets('Statement')

    ws.Range("H1").Value = NameOfDealer
    ws.Range("H2").Value = "Dealer Number: " + NumberOfDealer

    # Add Earning Amount to the cell for BHPH
    if dealer in BHPHList:
        remit = ws.Range("A44", "A53").Value
        remitPeriod = []

        for dt in remit:
            try:
                period = int(dt[0])
            except:
                period = dt[0]
            remitPeriod.append(str(period))

        startRow = 44
        startCol = 7
        
        # @p = the remit period shown in the report file.
        for p in remitPeriod:
            if p in list(earning.PostedDate):
                ws.Cells(startRow, startCol).Value = earning[earning.PostedDate == p].Amount.values[0]
            startRow += 1
    else:
        try:
            ws.Range("G23").Value = earning.iloc[-1, 1]
            ws.Range("H23").Value = earning.Amount.sum()
        except:
            ws.Range("G23").Value = 0
            ws.Range("H23").Value = 0
        
    # Select sheets for each kind of data
    ws1 = wb.Worksheets('Reserves')
    ws2 = wb.Worksheets('Cancels')
    ws3 = wb.Worksheets('Claims')
    ws4 = wb.Worksheets('Sheet1')
    
    # Import all data into each worksheets.
    ws1.Range(ws1.Cells(5,1), ws1.Cells(4 + len(contract.index), len(contract.columns))).Value = contract.values
    ws1.ListObjects('Table11').ShowTotals = True
    ws1.Columns("A:N").AutoFit()
    ws1.PageSetup.PrintArea = '$A$1:$J$' + str(5 + len(contract.values))
    ws2.Range(ws2.Cells(5,1), ws2.Cells(4 + len(cancel.index), len(cancel.columns))).Value = cancel.values
    ws2.ListObjects('Table12').ShowTotals = True
    ws2.Columns("A:N").AutoFit()
    ws2.PageSetup.PrintArea = '$A$1:$J$' + str(5 + len(contract.values))
    ws3.Range(ws3.Cells(6,1), ws3.Cells(5 + len(claim.index), len(claim.columns))).Value = claim.values
    ws3.ListObjects('Table13').ShowTotals = True
    ws3.Columns("A:N").AutoFit()
    ws3.PageSetup.PrintArea = '$A$1:$L$' + str(6 + len(contract.values))
    ws4.Range(ws4.Cells(27,1), ws4.Cells(26 + len(product.index), len(product.columns))).Value = product.values
    
    # Add BHPH data if the dealer participate BHPH
    if dealer in BHPHList:
        bhphWb = xl.Workbooks.Open(bhphPath)
        bhphWs = bhphWb.Worksheets(dealerName)
        bhphWs.Copy(Before = wb.Worksheets(4))
        
        ws5 = wb.Worksheets('Breakdown')
        ws6 = wb.Worksheets(dealerName)
        
        if len(dealerEarning) > 1:
            length = len(dealerEarning.index)-1
            times = int(length / 3)
            lastTime = int(length % 3)
            i = 0
            
            for i in range(times):
                ws5.Rows(str(16+3*i)+":"+str(15+3*(i+1))).Insert()
            
            if lastTime != 0:
                ws5.Rows(str(16+3*times)+":"+str(15+3*times+lastTime)).Insert()
        
        ws5.Range(ws5.Cells(16,1), ws5.Cells(15+len(dealerEarning.index), 1)).Value = pd.DataFrame(dealerEarning.Name).values
        ws5.Range(ws5.Cells(16,11), ws5.Cells(15+len(dealerEarning.index), 11)).Value = pd.DataFrame(dealerEarning.Amount).values
        ws5.Range(ws5.Cells(23+len(dealerEarning.index),1), ws5.Cells(23+len(dealerEarning.index)+len(productEarning.index), 1)).Value = pd.DataFrame(productEarning.ProductCode).values
        ws5.Range(ws5.Cells(23+len(dealerEarning.index),11), ws5.Cells(23+len(dealerEarning.index)+len(productEarning.index), 11)).Value = pd.DataFrame(productEarning.Amount).values
        ws6.Name = "BHPH"
    
    wb.Close(True)
    
    print(earning)
    print("The sum of earnings is: " + str(earning.Amount.sum()))
    
conn.close()

PgSQL Connected
  PostedDate  Amount
0       2019    3.10
1     2020 1   29.50
2     2020 2    3.68
3     2020 3   14.46
4     2020 4   18.68
5     2020 5   45.91
6     2020 6   44.37
7     2020 7   56.51
8     2020 8   67.65
9     2020 9  232.95
The sum of earnings is: 516.81


#### Create the report based on dealer and reinsurer

In [37]:
conn = psycopg2.connect(dbname='warehouse', user='chris', password='dowc_123', host='3.221.58.110', port='5432')
print('PgSQL Connected')

for dealer in dealerList:
    # SQL Query
    cancel_sql = """SELECT DISTINCT
                        "Dealer", "CashTransactionId", "SentDate"::text, "ReceivedDate"::text, "PostedDate"::TEXT,
                        "CurrentContractStatus", "DisbursementType", "ContractNumber", "Product", -1*sum("Amount"), "EffectiveDate"::TEXT
                    FROM
                        chris."tbl_WHCancel" whc
                    WHERE
                        "Dealer" ~* '{}'
                        AND COALESCE("ListReinsurerNames", 'No Reinsurer') ~* '{}'
                        AND "PostedDate" IS NOT NULL
                        AND CASE
                                WHEN "CancelMethod" = 'PRORATED' THEN "DisbursementType" = 'Reserve'
                                WHEN "CancelMethod" = 'FULL REFUND' THEN "DisbursementType" ~* 'reserve'
                            END
                    GROUP BY
                        1,2,3,4,5,6,7,8,9,11
                    ORDER BY
                        5,1,8;""".format(dealer, dealerReinsurer[dealer] + "|All WHS Cession")

    contract_sql = """SELECT
                            vr."Dealer", vr."CashTransactionId", vr."EffectiveDate"::text, vr."SentDate"::text, vr."PostedDate"::TEXT,
                            vr."TransactionTypeName", vr."DisbursementType", vr."ContractNumber", vr."Product", SUM(vr."Amount") AS "Reserve", vr."ProductCode", vr."ProductType"
                        FROM
                            chris."tbl_Reserve" vr
                        WHERE
                            vr."Dealer" ~* '{}'
                            AND COALESCE("ListReinsurerNames", 'No Reinsurer') ~* '{}'
                            AND vr."TransactionTypeName" !~* 'Cancel|Suspend|Pending Reinstatement'
                            AND vr."PostedDate" IS NOT NULL
                            AND vr."ReceivedDate" IS NOT NULL
                            AND vr."DisbursementType" ~* 'reserve'
                        GROUP BY
                            1,2,3,4,5,6,7,8,9,11,12
                        ORDER BY
                            5,1,8;""".format(dealer, dealerReinsurer[dealer] + "|All WHS Cession")

    claim_sql = """SELECT DISTINCT
                        vc."Dealer", vc."CashTransactionId", vc."SentDate"::text, vc."ReceivedDate"::text, vc."PostedDate"::text, vc."TC", vc."ClaimNumber",
                        vc."Status", vc."SubStatus", vc."Contract", vc."Product", vc."Amount", vc."ProductCode"
                    FROM
                        chris."tbl_Claim" vc
                    WHERE
                        vc."Dealer" ~* '{}'
                        AND COALESCE("ListReinsurerNames", 'No Reinsurer') ~* '{}'
                        AND vc."SubStatus" ~* 'FULLY PAID'
                        AND vc."PostedDate" IS NOT NULL
                    ORDER BY
                        5,1,10;""".format(dealer, dealerReinsurer[dealer] + "|All WHS Cession (Inhouse)")

     #  Read data into Dataframe
    cancel = pd.read_sql_query(cancel_sql, conn)
    contract = pd.read_sql_query(contract_sql, conn)
    claim = pd.read_sql_query(claim_sql, conn)
    
    # Select the date to be used for the postedDate
    dt = datetime.date(int(year), int(month), 1)
    dt = dt + relativedelta(months=1) - relativedelta(days=1)
    maxDate = str(dt)
    
    # Get the aggregate earning data   
    earning_sql = """SELECT
                        CASE
                            WHEN p."PostedDate" < '2020-1-1' THEN date_part('year', p."PostedDate")::TEXT
                            ELSE CONCAT(date_part('year', p."PostedDate") , ' ', date_part('month', p."PostedDate"))
                        END AS "PostedDate", sum(er."EarnedAmount"::NUMERIC) AS "Amount"
                    FROM
                        ssasearnedreserve er
                    JOIN ssascontractdimension con
                        ON con."ContractId" = er."fxContractId"
                    JOIN ssasdealerdimension deal
                        ON deal."DealerId" = con."fxSoldByDealerId"
                    LEFT JOIN ssaspostinghistorydimension p
                        ON p."PostingHistoryId" = er."fxPostingHistoryId"
                    WHERE
                        deal."Name" ~* '{}'
                        AND COALESCE((regexp_split_to_array("ListReinsurerNames", ','))[array_upper(regexp_split_to_array("ListReinsurerNames", ','), 1)], 'No Reinsurer') ~* '{}'
                        AND p."PostedDate" <= '{}'
                    GROUP BY
                        1
                    ORDER BY
                        1""".format(dealer, dealerReinsurer[dealer] + "|All WHS Cession (Inhouse)", maxDate)
    
    earning = pd.read_sql_query(earning_sql, conn)
    
    # Get the detailed earning data
    earningDetails_sql = """SELECT
                            deal."Name", con."ProductCode", sum(er."EarnedAmount"::NUMERIC) AS "Amount"
                        FROM
                            ssasearnedreserve er
                        JOIN ssascontractdimension con
                            ON con."ContractId" = er."fxContractId"
                        JOIN ssasdealerdimension deal
                            ON deal."DealerId" = con."fxSoldByDealerId"
                        JOIN dim_reinsurer_vw re
                            ON re."ContractId" = con."ContractId"
                        LEFT JOIN ssaspostinghistorydimension p
                            ON p."PostingHistoryId" = er."fxPostingHistoryId"
                        WHERE
                            deal."Name" ~* '{}'
                            AND COALESCE("ListReinsurerNames", 'No Reinsurer') ~* '{}'
                            AND p."PostedDate" <= '{}'
                        GROUP BY
                            1,2
                        HAVING
                            sum(er."EarnedAmount"::NUMERIC) <> 0
                        ORDER BY
                            1""".format(dealer, dealerReinsurer[dealer] + "|All WHS Cession (Inhouse)", maxDate)
    earningDetails = pd.read_sql_query(earningDetails_sql, conn)
    
    dealerEarning = earningDetails[['Name', 'Amount']].groupby('Name', as_index=False, sort=True).sum()
    productEarning = earningDetails[['ProductCode', 'Amount']].groupby('ProductCode', as_index=False).sum()
    
    # Used to count the ITD number of the product
    countContract = contract[['ProductType', 'ContractNumber']][~contract['ContractNumber'].isin(cancel.ContractNumber)].groupby('ProductType', as_index=False).count()
    product = countContract.rename(columns = {'ContractNumber': 'Count'})
    product.sort_values(by = ['Count'], ascending=False, inplace=True)

    # Create the dealer name used in the report
    if dealer == 'Heards Enterprise Inc.|Eagle Motors|National Advance Auto|Risner Auto Group|Iron City Imports, LLC|Wiltshire Auto Sales, LLC|Auto Arena|Tri City Motors' \
        or dealer == 'M and I|Crown|Birmingham Auto|Garner|P&M Auto|Lanier Automotive|Mardi Gras Motors|Oldham Motor|Peak Auto Sales|Perfect Ride|Wright Automotive':
        dealerName = 'PPT'
    elif dealer == "Dan O''Brien|Infiniti of Hanover":
        dealerName = 'Dan Obrien'
    elif dealer == 'AMG Auto|Parkwood Auto Mall':
        dealerName = 'AMG & Parkwood'
    elif dealer == 'Jersey City Kia|Kia Of Englewood':
        dealerName = 'Nirvana Auto Group, LLC'
    elif dealer == 'El Paso Honda|Poe Toyota':
        dealerName = 'Poe Auto Group'
    else:
        dealerName = dealer
        
    # Create new file for the dealer's data
    new_file = 'C:/Users/Huijie Qu/Documents/Python Scripts/' + year + '.' + month + ' ' + dealerName + '.xlsx'
    bhphPath = 'C:/Users/Huijie Qu/Documents/Python Scripts/BHPH/2020.8 BHPH Reserve.xlsm'
    
    if dealer in BHPHList:
        shutil.copy2("C:/Users/Huijie Qu/Documents/Python Scripts/PPT BHPH.xlsx", new_file)
    else:
        shutil.copy2("C:/Users/Huijie Qu/Documents/Python Scripts/PPT.xlsx", new_file)

    NameOfDealer_sql = """SELECT "Name", "Number" FROM ssasdealerdimension s WHERE "Name" ~* '{}'""".format(dealerName)
    NameOfDealer_query = pd.read_sql_query(NameOfDealer_sql, conn)
    
    if NameOfDealer_query.empty:
        NameOfDealer = dealerName
        NumberOfDealer = ''
    else:
        NameOfDealer = NameOfDealer_query.Name[0]
        NumberOfDealer = NameOfDealer_query.Number[0]

    xl = MyWinCOM.Dispatch("Excel.Application")
    wb = xl.Workbooks.Open(new_file)
    ws = wb.Worksheets('Statement')

    ws.Range("H1").Value = NameOfDealer
    ws.Range("H2").Value = "Dealer Number: " + NumberOfDealer

    # Add Earning Amount to the cell for BHPH
    if dealer in BHPHList:
        remit = ws.Range("A44", "A53").Value
        remitPeriod = []

        for dt in remit:
            try:
                period = int(dt[0])
            except:
                period = dt[0]
            remitPeriod.append(str(period))

        startRow = 44
        startCol = 7
        
        # @p = the remit period shown in the report file.
        for p in remitPeriod:
            if p in list(earning.PostedDate):
                ws.Cells(startRow, startCol).Value = earning[earning.PostedDate == p].Amount.values[0]
            startRow += 1
    else:
        try:
            ws.Range("G23").Value = earning.iloc[-1, 1]
            ws.Range("H23").Value = earning.Amount.sum()
        except:
            ws.Range("G23").Value = 0
            ws.Range("H23").Value = 0
        
    # Select sheets for each kind of data
    ws1 = wb.Worksheets('Reserves')
    ws2 = wb.Worksheets('Cancels')
    ws3 = wb.Worksheets('Claims')
    ws4 = wb.Worksheets('Sheet1')
    
    # Import all data into each worksheets.
    ws1.Range(ws1.Cells(5,1), ws1.Cells(4 + len(contract.index), len(contract.columns))).Value = contract.values
    ws1.ListObjects('Table11').ShowTotals = True
    ws1.Columns("A:N").AutoFit()
    ws1.PageSetup.PrintArea = '$A$1:$J$' + str(5 + len(contract.values))
    ws2.Range(ws2.Cells(5,1), ws2.Cells(4 + len(cancel.index), len(cancel.columns))).Value = cancel.values
    ws2.ListObjects('Table12').ShowTotals = True
    ws2.Columns("A:N").AutoFit()
    ws2.PageSetup.PrintArea = '$A$1:$J$' + str(5 + len(cancel.values))
    ws3.Range(ws3.Cells(6,1), ws3.Cells(5 + len(claim.index), len(claim.columns))).Value = claim.values
    ws3.ListObjects('Table13').ShowTotals = True
    ws3.Columns("A:N").AutoFit()
    ws3.PageSetup.PrintArea = '$A$1:$L$' + str(6 + len(claim.values))
    ws4.Range(ws4.Cells(27,1), ws4.Cells(26 + len(product.index), len(product.columns))).Value = product.values
    
    # Add BHPH data if the dealer participate BHPH
    if dealer in BHPHList:
        bhphWb = xl.Workbooks.Open(bhphPath)
        bhphWs = bhphWb.Worksheets(dealerName)
        bhphWs.Copy(Before = wb.Worksheets(4))
        
        ws5 = wb.Worksheets('Breakdown')
        ws6 = wb.Worksheets(dealerName)
        
        if len(dealerEarning) > 1:
            length = len(dealerEarning.index)-1
            times = int(length / 3)
            lastTime = int(length % 3)
            i = 0
            
            for i in range(times):
                ws5.Rows(str(16+3*i)+":"+str(15+3*(i+1))).Insert()
            
            if lastTime != 0:
                ws5.Rows(str(16+3*times)+":"+str(15+3*times+lastTime)).Insert()
        
        ws5.Range(ws5.Cells(16,1), ws5.Cells(15+len(dealerEarning.index), 1)).Value = pd.DataFrame(dealerEarning.Name).values
        ws5.Range(ws5.Cells(16,11), ws5.Cells(15+len(dealerEarning.index), 11)).Value = pd.DataFrame(dealerEarning.Amount).values
        ws5.Range(ws5.Cells(23+len(dealerEarning.index),1), ws5.Cells(23+len(dealerEarning.index)+len(productEarning.index), 1)).Value = pd.DataFrame(productEarning.ProductCode).values
        ws5.Range(ws5.Cells(23+len(dealerEarning.index),11), ws5.Cells(23+len(dealerEarning.index)+len(productEarning.index), 11)).Value = pd.DataFrame(productEarning.Amount).values
        ws6.Name = "BHPH"
    
    wb.Close(True)
    
    print(earning)
    print("The sum of earnings is: " + str(earning.Amount.sum()))
    
conn.close()

PgSQL Connected
  PostedDate  Amount
0     2020 1    0.00
1     2020 2    0.00
2     2020 3    0.00
3     2020 4    0.00
4     2020 5    0.00
5     2020 6    0.00
6     2020 7  785.94
7     2020 8  396.84
8     2020 9  384.02
The sum of earnings is: 1566.8


In [35]:
wb = xl.Workbooks.Open('C:/Users/Huijie Qu/Documents/Python Scripts/2020.9 Parkwood Auto.xlsx')
ws1 = wb.Worksheets('Reserves')
ws1.PageSetup.PrintArea = '$A$1:$J$' + str(5 + len(contract.values))

In [6]:
conn = psycopg2.connect(dbname='warehouse', user='chris', password='dowc_123', host='3.221.58.110', port='5432')
print('PgSQL Connected')

sql = """SELECT
            deal."Name", p."PostedDate", sum(er."EarnedAmount"::NUMERIC) AS "Amount"
        FROM
            ssasearnedreserve er
        JOIN ssascontractdimension con
            ON con."ContractId" = er."fxContractId"
        JOIN ssasdealerdimension deal
            ON deal."DealerId" = con."fxSoldByDealerId"
        LEFT JOIN ssaspostinghistorydimension p
            ON p."PostingHistoryId" = er."fxPostingHistoryId"
        WHERE
            concat(deal."Name", con."ProductCode") ~* '{}'
        GROUP BY
            1, 2
        HAVING
            sum(er."EarnedAmount"::NUMERIC) <> 0
        ORDER BY
            2, 1
        """.format(dealerProduct['PPT'])
df = pd.read_sql_query(sql, conn)

PgSQL Connected


In [7]:
df.to_csv('test.csv')